 ## Geometric mean
 _CYBR 304 & MATH 420_ <br>
 Spring 2024 <br>

To write robust numerical code, we must be aware of numerical over and underflow, numerical accuracy, and we need to give "edge cases" special attention. This Julia worksheet illustrates some of these issues for computing the geometric mean. In this context, edge cases include the geometric sum of the empty list as well as how to handle Julia's extended real numbers `Inf` and `NaN`.

The _geometric mean_ $\mathrm{GM}$ of nonnegative numbers $x_1, x_2, \dots, x_n$ is the nth root of their product. Specifically
$$
   \mathrm{GM}(x_1, x_2, \dots x_n) = \left( \prod_{k=1}^n x_k \right)^{1/n}.
$$
The geometric mean is often used in finance to find investment returns, and it has other applications in engineering as well. In this worksheet, we'll write Julia code that evaluates the geometric mean, paying particular attention to overflow, underflow, and accuracy.

One property of the geometric mean is that $\mathrm{GM}(x,x) = x$ is an identity for all nonnegative numbers $x$. We'll use this identity as a test for our Julia code. Another property of the geometric mean is that any one of the numbers $x_1, x_2, \dots, x_n$ is zero, the geometric mean of these numbers is zero.

When the input array is empty, we need to decide what to do. The convention in mathematics is that an empty product is the multiplicative identity, making the geometric mean of an empty list is the zeroth root of one. But the zeroth root of one is indeterminate, so we should write or code so that geometric mean of an empty list throws
an error.

In [2]:
"""
    geometric_mean(a::Array)

Compute the geometric mean of the elements in the input array `a`. 
"""
function geometric_mean(a::Array)
   isempty(a) && throw(ArgumentError("The input to geometric_mean must be a nonempty array."))
   prod(a)^(1/length(a)) 
end;

Three simple tests show that our function works OK.

In [4]:
(geometric_mean([5,20]), sqrt(5*20))

(10.0, 10.0)

In [5]:
(geometric_mean([1,2,3,4]), (1*2*3*4)^(1/4))

(2.2133638394006434, 2.2133638394006434)

In [6]:
(geometric_mean([0,1,2,3,4]), (0*1*2*3*4)^(1/5))

(0.0, 0.0)

When one or more inputs are `Inf` and all other inputs are positive, the result is `Inf`. This is the correct behavior.

In [8]:
geometric_mean([Inf,4,5])

Inf

But if one input is zero and one is `Inf`, the result is `NaN`.  This is appropriate.

In [10]:
geometric_mean([Inf, 0])

NaN

Possibly, an input that both `Inf` and `Inf` could be handled more gracefully:

In [12]:
geometric_mean([Inf, -Inf])

LoadError: DomainError with -Inf:
Exponentiation yielding a complex result requires a complex argument.
Replace x^y with (x+0im)^y, Complex(x)^y, or similar.

In [13]:
geometric_mean([])

LoadError: ArgumentError: The input to geometric_mean must be a nonempty array.

Our function can needlessly underflow and return zero. We should fix this. An example--according to the identity $\mathrm{GM}(x,x) = x$, the result of this should be 
$2^{-538}$, not zero

In [15]:
geometric_mean([2^(-538), 2^(-538)])

0.0

Possibly, our code should throw an Argument Error when an input is negative, but it doesn't, so a user can get an error message and a stack trace that isn't easy to understand:

In [17]:
geometric_mean([-1,-2,-4])

LoadError: DomainError with -8.0:
Exponentiation yielding a complex result requires a complex argument.
Replace x^y with (x+0im)^y, Complex(x)^y, or similar.

When an input is a complex number, our code works OK

In [19]:
geometric_mean([2+im, 6.7 + im])

3.7113075237539723 + 1.172093654906827im

But a bit more testing shows that our code can overflow giving suboptimal results; for example, a better value for this is 1.0e155, not `Inf`

In [21]:
geometric_mean([1.0e155, 1.0e155])

Inf

This result violates the identity $\mathrm{GM}(x,x) = x$.  We really should do better.  A typical way to fix this overflow problem is to use the fact that the logarithm of a product of positive numbers is the sum of the logarithms.  This gives an alternative formula for the geometric sum
$$
   \mathrm{GM}(x_1, x_2, \dots, x_n) = \exp \left (\frac{1}{n}  \left( \sum_{k=1}^n \ln(x_k) \right) \right)
$$
Alternatively, we can use the base-two logarithm and base two exponentiation. For floating point numbers in binary form, this is possibly the most natural choice.

A simple implementation of this method is

In [23]:
"""
    geometric_mean_log(a::Array)

Compute the geometric mean of the elements in the input array `a` using a logarithmic 
transformation to avoid overflow. When the array `a` is empty throw an ArgumentError.
"""
function geometric_mean_log(a::Array)
    isempty(a) && throw(ArgumentError("The input to geometric_mean_log must be a nonempty array."))
    exp2(sum(map(log2,a))/length(a))
end;

In Julia `log` is the natural logarithm, and `log2` is the base two logarithm. Many computer languages use `ln` for the natural logarithm. The Julia function `map` applies a function to each member of an array and the Julia function `sum` adds the members of an array. Finally, `exp2` is the function $x \mapsto 2^x$.

Here are a few simple tests:

In [25]:
(geometric_mean_log([5,20]), sqrt(5*20))

(10.000000000000002, 10.0)

In [26]:
(geometric_mean_log([1,2,3,4]), (1*2*3*4)^(1/4))

(2.213363839400643, 2.2133638394006434)

We have resolved the overflow problem, but arguably our function isn't as accurate as it might be; for example

In [28]:
geometric_mean_log([1.0e155, 1.0e155])

1.000000000000028e155

Because $\log(0)$ is undefined, you might think that `geometric_mean_log` misbehaves when one or more argument is zero, but it doesn't.

In [30]:
geometric_mean_log([0,1,2,3])

0.0

To see what happens, we can work through the calculation one step at a time

In [32]:
x = map(log,[0,1,2,3])

4-element Vector{Float64}:
 -Inf
   0.0
   0.6931471805599453
   1.0986122886681098

In [33]:
x = sum(x)

-Inf

In [34]:
x = x/4

-Inf

In [35]:
exp(x)

0.0

In Julia log(0) = -Inf and exp(-Inf) = 0.


Also, the code has resolved the underflow problem; for example

In [38]:
geometric_mean_log([2^(-538), 2^(-538), 2^(-538)])

1.1113793747425387e-162

In [39]:
sum(map(log2, Float64[]))

0.0

Let's check that the code throws an Argument error when the input is an empty array.

In [41]:
geometric_mean_log([])

LoadError: ArgumentError: The input to geometric_mean_log must be a nonempty array.

In [42]:
"""
    geometric_mean2(a::AbstractArray{<:AbstractFloat})

Compute the geometric mean of the elements in the input array `a`.  When the array is empty, throw an error.
"""
function geometric_mean2(a::AbstractArray{<:AbstractFloat})
    isempty(a) && throw(ArgumentError("The input to geometric_mean must be a nonempty array."))
    n = length(a)
    e = 0
    s = one(eltype(a))
    for x in a
      s *= x
      e += exponent(s)
      s = significand(s)
    end
     exp2(e/n)*s^(1/n)
end;   

Let's do some simple checks for empty arrays, overflow, and underflow

In [44]:
geometric_mean2(Float16[])

LoadError: ArgumentError: The input to geometric_mean must be a nonempty array.

In [45]:
geometric_mean2([1.0e155, 1.0e155])

1.0e155

In [46]:
geometric_mean2([1.0e308, 1.0e308, 1.0e308])

1.0e308

There is a standard Julia package that has code for the geometric mean.  To use it, we need to use the package manager to download and install it. Once we have done that one time, to use the package, we only need to load it. Additionally, to compare running times, we'll use the `BenchmarkTools` package.

In [48]:
using StatsBase, BenchmarkTools

Specifically, the function name is `geomean`.  It handles the underflow and overflow tests OK, but arguably, and overflow test shows that `geomean` isn't as accurate as it might be

In [50]:
geomean([2^(-538), 2^(-538)])

1.1113793747425612e-162

These two results should be the same, but they are not

In [52]:
(geomean([exp2(155) for k=1:10^7]) , exp2(155))

(4.5671926166570814e46, 4.567192616659072e46)

Our code returns an accurate value

In [54]:
(geometric_mean2([exp2(155) for k=1:10^7]) , exp2(155))

(4.567192616659072e46, 4.567192616659072e46)

Julia throws a domain error for `exponent(Inf)`, so our unlike our other two implementations, this code cannot handle input arrays that have `Inf` as a member. A robust implementation would trap for this case

In [56]:
geometric_mean2([2,20,Inf])

LoadError: DomainError with Inf:
Cannot be NaN or Inf.

But the geomean function properly returns Inf for this case


In [58]:
geomean([2,20,Inf])

Inf

And a few quick checks too

In [60]:
geometric_mean2([5.0,20.0])

10.0

In [61]:
geometric_mean2([5.0,20.0, 4.0]), (5.0 * 20.0 * 4.0)^(1/3)

(7.368062997280774, 7.368062997280773)

But the `geomean` function properly returns `Inf` for this case

Let's run a race between these two functions. We'll time them on a ten-million element array of random numbers. 

Keep in mind that Julia has a just in time compiler, so the first time we run the code, it's slow because the Julia compiler has to be used. The timing function in `BenchmarkTools` runs the test multiple times and accounts for the time for compilation.

One test shows that our code `geometric_mean` is about twice as fast as `geomean`.

In [64]:
L = rand(Float64,10^7);

In [65]:
@btime x = geometric_mean2(L)

  35.622 ms (1 allocation: 16 bytes)


0.36799148422814704

In [66]:
@btime y = geomean(L)

  86.617 ms (1 allocation: 16 bytes)


0.3679914842281471

For the fastest speed, we should stick with our first implementation, but it is prone to over and underflow, it is much faster than all the other methods.

In [68]:
@btime x = geometric_mean(L)

  4.349 ms (1 allocation: 16 bytes)


0.0

In [110]:
using Pandoc